# Домашнее задание 3

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
def clean_russian_text(text):
    clean_text = re.sub(r'[^а-яА-Я ]+', ' ', text)
    clean_text = re.sub("\s\s+", " ", clean_text)
    return clean_text.strip().lower()

def count_frequencies(array):
    return dict(pd.value_counts(array, normalize=True))

In [3]:
class Encoder:
    def __init__(self, true_to_enc=None, enc_to_true=None):
        self.true_to_enc = true_to_enc
        self.enc_to_true = enc_to_true

    def init_random(self, array):
        true_values = sorted(set(array))
        encoded_values = true_values.copy()
        np.random.shuffle(encoded_values)
        self.true_to_enc = dict(zip(true_values, encoded_values))
        self.enc_to_true = dict(zip(encoded_values, true_values))
        return self

    def encode(self, array):
        return [self.true_to_enc[x] for x in array]

    def decode(self, array):
        return [
            self.enc_to_true[x] if x in self.enc_to_true
            else " " * len(x)
            for x in array
        ]


def fit_by_freq(true_freq, enc_freq):
    enc_to_true = {}
    for k, freq in enc_freq.items():
        distances = [
            np.abs(t_freq - freq) for t_freq in true_freq.values()
        ]
        enc_to_true[k] = list(true_freq)[np.argmin(distances)]
    true_to_enc = {v: k for k, v in enc_to_true.items()}
    return enc_to_true, true_to_enc


def fit_by_freq_order(true_freq, enc_freq, max_size=np.inf):
    min_len = min(len(true_freq), len(enc_freq), max_size)
    true_sorted = list(true_freq.items())
    enc_sorted = list(enc_freq.items())

    true_sorted.sort(key=lambda x: x[1])
    enc_sorted.sort(key=lambda x: x[1])

    true_sorted_keys = [x[0] for x in true_sorted][-min_len:]
    enc_sorted_keys = [x[0] for x in enc_sorted][-min_len:]

    enc_to_true = dict(zip(enc_sorted_keys, true_sorted_keys))
    true_to_enc = dict(zip(true_sorted_keys, enc_sorted_keys))

    return enc_to_true, true_to_enc

def split_n_grams(text, n=2, step=1):
    n_grams = []
    for i in range(0, len(text) + 1 - n, step):
        n_grams.append(text[i: i + n])

    return n_grams

def text_accuracy(text1, text2):
    return (np.array(list(text1)) == np.array(list(text2))).mean()

# Подготовка данных

In [5]:
with open("./texts/WarAndPeace.txt") as f:
    war_and_peace = f.read()

with open("./texts/AnnaKarenina.txt") as f:
    anna_karenina = f.read()

In [6]:
war_and_peace_prep = clean_russian_text(war_and_peace)
anna_karenina_prep = clean_russian_text(anna_karenina)

В качестве тренировочных данных будем использовать войну и мир.

In [7]:
true_words_freq = count_frequencies(list(war_and_peace_prep))

In [8]:
true_words_freq

{' ': 0.16243574314049278,
 'о': 0.09528829677728332,
 'а': 0.07029614909768286,
 'е': 0.0661134279343577,
 'и': 0.055725041282991404,
 'н': 0.05460705744788703,
 'т': 0.04760994026016837,
 'с': 0.04373664716803344,
 'л': 0.04241341456208931,
 'в': 0.038599208237317335,
 'р': 0.03820425984494388,
 'к': 0.030053395778716945,
 'д': 0.02548039096781015,
 'м': 0.024785343993830096,
 'у': 0.024029655337556483,
 'п': 0.02153090704407562,
 'я': 0.01940067358914794,
 'г': 0.017379284179362547,
 'ь': 0.016323496941482333,
 'ы': 0.015911444484872233,
 'з': 0.014930293163661016,
 'б': 0.014476258003924606,
 'ч': 0.011427069825009874,
 'й': 0.009656021719051751,
 'ж': 0.008489835521098641,
 'ш': 0.007914516996775107,
 'х': 0.007152608680779075,
 'ю': 0.005434427682461492,
 'ц': 0.0033881596337864356,
 'э': 0.002532956421954155,
 'щ': 0.002354141204934678,
 'ф': 0.0018798921511004132,
 'ъ': 0.0004400409253609735}

В качестве текста, который будем расшифровывать, возьмем произведение Анна Каренина

In [28]:
test_text = anna_karenina_prep
test_text[:1000]

'анна каренина один из самых знаменитых романов льва толстого начинается ставшей афоризмом фразой все счастливые семьи похожи друг на друга каждая несчастливая семья несчастлива по своему это книга о вечных ценностях о любви о вере о семье о человеческом достоинстве лев толстойроман широкого дыхания часть первая лев толстой анна каренина роман широкого дыхания анна каренина поразила современников вседневностью содержания необычайная свобода раскованность повествования удивительно сочетались в этом романе с цельностью художественного взгляда автора на жизнь он выступал здесь как художник и мыслитель и назначение искусства видел не в том чтобы неоспоримо разрешить вопрос а в том чтобы заставить любить жизнь в бесчисленных никогда не истощимых всех ее проявлениях в е годы один маститый писатель по видимому гончаров сказал достоевскому это вещь неслыханная это вещь первая кто у нас из писателей может поравняться с этим а в европе кто представит хоть что нибудь подобное ф м достоевский нахо

Вместо рандомно зашифровываного текста будем использовать исходный текст, а декодер должен будет угадать, что а -> a, б -> б, и так далее. Так нам будет удобнее понимать, какие буквы совпали, а какие нет.

In [29]:
test_text_enc = test_text

Посчитаем частоты букв в зашифрованном тексте.

In [30]:
test_text_enc_list = list(test_text_enc)
enc_freq = count_frequencies(list(test_text_enc))

### Декодируем с помощью fit_by_freq

Попробуем сопоставлять слова по частотам. Каждой букве из зашифрованного текста будем сопоставлять букву из тернировочного текста, которая наиболее близка ей по частоте.

In [31]:
enc_to_true, true_to_enc = fit_by_freq(true_words_freq, enc_freq)

decoder = Encoder(true_to_enc, enc_to_true)

decoded_text = "".join(decoder.decode(list(test_text_enc)))
decoded_text

'аииа какаиииа омии иб сауых биауаиитых коуаиов льва толстозо иабииаатсг ставхаж аъокибуоу ъкабож вса сбастливыа сауьи яохойи мкпз иа мкпза каймаг иасбастливаг сауьг иасбастлива яо своауп цто кииза о вабиых щаииостгх о любви о вака о сауьа о баловабаскоу мостоииства лав толстожкоуаи хикокозо мыхаииг басть яакваг лав толстож аииа какаиииа коуаи хикокозо мыхаииг аииа какаиииа яокабила совкауаиииков всамиавиостью сомакйаииг иаобыбажиаг свобома касковаииость яоваствоваииг пмивитальио собатались в цтоу коуаиа с щальиостью хпмойастваииозо вбзлгма автока иа йибиь ои выстпяал бмась как хпмойиик и уыслиталь и иабиабаииа искпсства вимал иа в тоу бтобы иаосяокиуо кабкахить воякос а в тоу бтобы баставить любить йибиь в басбислаииых иикозма иа истощиуых всах аа якогвлаиигх в а зомы омии уаститыж яисаталь яо вимиуоуп зоибаков скабал мостоавскоуп цто ващь иаслыхаииаг цто ващь яакваг кто п иас иб яисаталаж уойат яокавигтьсг с цтиу а в авкояа кто якамставит хоть бто иибпмь яомобиоа ъ у мостоавскиж иахо

In [32]:
enc_to_true

{' ': ' ',
 'о': 'о',
 'е': 'а',
 'а': 'а',
 'н': 'и',
 'и': 'и',
 'т': 'т',
 'с': 'с',
 'л': 'л',
 'в': 'в',
 'р': 'к',
 'к': 'к',
 'д': 'м',
 'м': 'у',
 'у': 'п',
 'п': 'я',
 'я': 'г',
 'ь': 'ь',
 'ы': 'ы',
 'г': 'з',
 'б': 'б',
 'ч': 'б',
 'з': 'б',
 'ж': 'й',
 'й': 'ж',
 'ш': 'х',
 'х': 'х',
 'ю': 'ю',
 'э': 'ц',
 'щ': 'щ',
 'ц': 'щ',
 'ф': 'ъ',
 'ъ': 'ъ'}

In [33]:
text_accuracy(test_text, decoded_text)

0.652314225654332

65% символов расшифровываются правильно, однако смысл текста сложно понять

### Декодируем с помощью fit_by_order

Попробуем теперь декодировать другим способом. Отстортируем буквы в тренировочном и закодированном текстах по частотам и после этого сопоставим эти два отсортированных списка букв: самая частая буква в закодированном тексте будет расшифровываться как самая частая буква в тренировачном тексте. Вторая по частоте буква в закодированном тексте будет расшифровываться как вторая по частоте буква в теронировачном тексте, и так далее.

In [34]:
enc_to_true, true_to_enc = fit_by_freq_order(true_words_freq, enc_freq)

decoder = Encoder(true_to_enc, enc_to_true)

decoded_text = "".join(decoder.decode(list(test_text_enc)))
print(decoded_text[:1000])


print(f"accuracy: {text_accuracy(test_text, decoded_text)}")

еиие кераиние одни нч семьх чиемаинтьх ромеиов лгве толстоыо иебниеатся стевшаж ефорнчмом фречож вса сбестлнвьа самгн похойн друы ие друые кейдея иасбестлнвея самгя иасбестлнве по своаму цто киные о вабиьх щаииостях о люзвн о вара о самга о баловабаском достониства лав толстожромеи шнрокоыо дьхеиня бестг парвея лав толстож еиие кераиние ромеи шнрокоыо дьхеиня еиие кераиние поречнле соврамаиинков всадиавиостгю содарйеиня иаозьбежиея свозоде ресковеииостг поваствовеиня уднвнталгио собателнсг в цтом ромеиа с щалгиостгю худойастваииоыо вчыляде евторе ие йнчиг ои вьступел чдасг кек худойинк н мьслнталг н иечиебаина нскусстве вндал иа в том бтозь иаоспорнмо речрашнтг вопрос е в том бтозь честевнтг люзнтг йнчиг в засбнслаииьх инкоыде иа нстоэнмьх всах аа проявлаинях в а ыодь одни местнтьж пнсеталг по внднмому ыоиберов скечел достоавскому цто ваэг иасльхеииея цто ваэг парвея кто у иес нч пнсеталаж мойат поревиятгся с цтнм е в авропа кто прадстевнт хотг бто инзудг подозиоа ф м достоавскнж иеход

Качество такого метода оказалось хуже по старвнению с предыдущим

## Биграммы

In [38]:
## Из сопоставления биграмм делает сопоставление слов

def bigrams_to_word_map(enc_to_true_bigrams):
    enc_to_true = []
    for enc, true in enc_to_true_bigrams.items():
        enc_to_true.append([enc[0], true[0]])
        enc_to_true.append([enc[1], true[1]])

    enc_to_true_letters = dict(
        pd.DataFrame(enc_to_true, columns=["enc", "true"])
        .groupby("enc")
        .true.agg(pd.Series.mode)
    )
    enc_to_true_letters = {k: v[0] if isinstance(v, np.ndarray) else v for k, v in enc_to_true_letters.items()}

    true_to_enc_letters = {v: k for k, v in enc_to_true_letters.items()}


    return enc_to_true_letters, true_to_enc_letters

Разобьем тренировачный текст на биграммы и посчитаем их частоты

In [35]:
true_words_freq_bigrams = count_frequencies(split_n_grams(war_and_peace_prep, 2, 1))
len(true_words_freq_bigrams)

764

Сделаем то же самое с зашифрованным текстом

In [36]:
test_text_enc_bigrams = split_n_grams(test_text_enc, 2, 2)
enc_freq_bigrams = count_frequencies(test_text_enc_bigrams)

Попробуем расшифровать наш текст, используюя частоты биграмм(первый метод)

In [41]:
enc_to_true_bigrams, true_to_enc_bigrams = fit_by_freq(true_words_freq_bigrams, enc_freq_bigrams)

enc_to_true_letters, true_to_enc_letters = bigrams_to_word_map(enc_to_true_bigrams)

decoder_bigrams = Encoder(true_to_enc_letters, enc_to_true_letters)
decoded_text = "".join(decoder_bigrams.decode(split_n_grams(anna_karenina_prep, 1, 1)))

decoded_text[:1000]

'а  а лар    а ол    з варшх з ар   ошх рора ор лсра оолвооао  ал  а овл воарр р адор зрор дразор рв  влавол рш  в рс  бохоа  лрта  а лртаа лаалал   влавол рал в рсл   влавол ра бо вро рт аоо л  аа о р л шх л   оволх о лррр  о р р  о в рс  о л лор л влор ловоо  вор  л р оолвооррора  р ролоао лшха  л лавос б ррал л р оолвоор а  а лар    а рора  р ролоао лшха  л а  а лар    а бораз ла ворр р    лор рв л  р овоср вол раа  л   оршлар ал вророла равлора  овос бор ворора  л тл р о лс о вол оал вс р аоор рора   в л лс овоср хтлоа вор   оао рзаллла ароора  а а з с о  ршвотбал зл вс лал хтлоа  л   ршвл о лс    аз ал      влтввора р л л    р оор лоорш   овбор ро разр р ос робров а р оор лоорш завоар ос лрр ос а з с р р вл вл   шх   лоала     воой ршх рв х    бролрл   лх р   аолш ол   раво ошр б вао лс бо р л рорт ао ларор влазал ловоо рвлорт аоо р йс   влшха  ал аоо р йс б ррал лоо т  ав  з б вао л р роа о борар лосвл в ао р а р  рроб  лоо бр лвоар о хоос лоо   ртлс болор о  д р ловоо рвл р  ахо

In [42]:
text_accuracy(test_text, decoded_text)

0.42734781026010443

В случае с биграммами качество получилось хуже чем при сравнении просто частот букв

## MCMC decoder

Попробуем расшифровать методом основанном на на MCMC-сэмплировании. Каждой букве  из закодированного текста будем сопоставлять рандомную букву из теринировачного текста и на основе логарифма правдоподобия принимать решение, делать следующий шаг или нет.

In [61]:
from tqdm import tqdm
from scipy.stats import bernoulli, binom, poisson


def metropolis_hastings_log_accept(l, l_new):
    if l_new > l:
        return True
    else:
        return np.random.rand() < (np.exp(l_new - l))


def cut_frequency(f_dict, top_n):
    series = pd.Series(f_dict).sort_values(ascending=False)[:top_n]
    return dict(series)


class MHDecoder:
    def __init__(self, text, n=1, freq_vocab_size=500):

        text_split = split_n_grams(text, n)
        self.true_letters = set(list(text))
        self.true_ngrams_freq = cut_frequency(count_frequencies(text_split), freq_vocab_size)
        self.enc_to_true = None
        self.num_accepted_steps = 0
        self.n = n

    def fit(self, text, n_steps=100):
        self.enc_text = text
        self.enc_letters = set(list(text))
        self._init_mapper()

        for _ in tqdm(range(n_steps)):
            self._make_step()


    def _init_mapper(self):
        mapper = {}
        true_letters = list(self.true_letters)
        for enc_letter in self.enc_letters:
            mapper[enc_letter] = np.random.choice(true_letters, 1)[0]

        self.enc_to_true = mapper
        self.log_likelihood = self._calculate_log_likelihood(mapper)

    def _change_mapper(self):
        mapper = self.enc_to_true.copy()
        key_to_change = np.random.choice(list(self.enc_letters), 1)[0]
        true_value = mapper[key_to_change]
        replace_variants = [x for x in self.true_letters if x != true_value]
        mapper[key_to_change] = np.random.choice(replace_variants, 1)[0]

        return mapper

    # def _change_mapper(self):
    #     mapper = self.enc_to_true.copy()
    #     keys_to_change = np.random.choice(list(self.enc_letters), 2, replace=False)
    #
    #     mapper[keys_to_change[0]], mapper[keys_to_change[1]] = mapper[keys_to_change[1]], mapper[keys_to_change[0]]
    #
    #     return mapper


    def _make_step(self):
        new_mapper = self._change_mapper()
        new_log_likelihood = self._calculate_log_likelihood(new_mapper)
        if metropolis_hastings_log_accept(self.log_likelihood, new_log_likelihood):
            self.enc_to_true = new_mapper.copy()
            self.log_likelihood = new_log_likelihood
            self.num_accepted_steps += 1

    def make_accepted_step(self):
        prev_num_accepted_steps = self.num_accepted_steps
        while prev_num_accepted_steps == self.num_accepted_steps:
            self._make_step()

    def _calculate_log_likelihood(self, mapper):
        decoded_text = "".join([mapper[x] for x in self.enc_text])
        decoded_text_split = split_n_grams(decoded_text, self.n)
        decoded_ngrams_freq = count_frequencies(decoded_text_split)

        log_likelihood = 0
        for true_ngram, true_gram_freq in self.true_ngrams_freq.items():
            decoded_freq = decoded_ngrams_freq.get(true_ngram) or 0
            # log_proba = np.log(
            #     binom.pmf(
            #         int(decoded_freq * len(decoded_text_split)),
            #         len(decoded_text_split),
            #         true_gram_freq
            #     ) + 0.0001
            # )

            log_proba = np.log(
                poisson.pmf(
                    int(decoded_freq * len(decoded_text_split)),
                    int(true_gram_freq * len(decoded_text_split))
                ) + 0.0001
            )
            log_likelihood += log_proba

        return log_likelihood

    # def _calculate_log_likelihood(self, mapper):
    #     decoded_text = "".join([mapper[x] for x in self.enc_text])
    #     decoded_text_split = split_n_grams(decoded_text, self.n)
    #     decoded_ngrams_freq = count_frequencies(decoded_text_split)
    #
    #
    #     log_likelihood = 0
    #     for true_ngram, true_gram_freq in self.true_ngrams_freq.items():
    #         decoded_freq = decoded_ngrams_freq.get(true_ngram) or 0
    #         log_proba = np.log(true_gram_freq + 0.0001) + np.log(decoded_freq + 0.0001)
    #         log_likelihood += log_proba
    #
    #     return log_likelihood


Запустим наш алгоритм на 50000  итераций на отрывке из произведения Анна Каренина

In [68]:
decoder = MHDecoder(war_and_peace_prep, n=2, freq_vocab_size=200)
decoder.fit(anna_karenina_prep[1_000:5_000], 50_000)
decoder.num_accepted_steps

100%|██████████| 50000/50000 [19:25<00:00, 42.91it/s]


1581

In [79]:
decoder.enc_to_true

{' ': 'о',
 'ф': 'с',
 'ч': 'д',
 'ж': 'р',
 'я': 'я',
 'р': 'н',
 'ц': 'ч',
 'й': 'т',
 'м': 'к',
 'п': 'р',
 'ь': 'о',
 'э': 'в',
 'д': 'г',
 'л': 'л',
 'о': 'а',
 'х': 'т',
 'ъ': 'б',
 'в': 'в',
 'ш': 'ш',
 'г': 'й',
 'к': 'м',
 'а': ' ',
 'т': 'ы',
 'н': 'б',
 'е': 'е',
 'и': 'и',
 'с': 'з',
 'у': 'у',
 'з': 'с',
 'ы': 'р',
 'ю': 'х',
 'щ': 'с',
 'б': 'т'}

In [80]:
true_to_enc = {v: k for k, v in decoder.enc_to_true.items()}

Попробуем декодировать текст

In [81]:
decoder_bigrams = Encoder(true_to_enc, decoder.enc_to_true)
decoded_text = "".join(decoder_bigrams.decode(split_n_grams(anna_karenina_prep, 1, 1)))

decoded_text[:1000]

' бб ом небиб оагибоисоз кртосб кебиыртонак баволов оыалзыайаоб диб еызяозы вшето санискакосн сатовзеозд зыливреозекоиоратариогнуйоб огнуй ом рг яобезд зылив яозекояобезд зылив ораозваекуовыаомбий оаоведбрточеббазыятоаолхтвиоаовенеоаозекоеоаоделаведезмакогазыаибзывеолевоыалзыатнак бошинамайаогрт бияод зыооренв яолевоыалзыато бб ом небиб онак бошинамайаогрт бияо бб ом небиб оран сил озавнекеббимавовзегбевбазыохозагенр бияобеатрд тб яозватаг он змав ббазыооравезывав бияоугивиыелобаозадеы лизоововыаконак беозочелобазыохотугарезывеббайаовсйляг о выан об орисбооабоврзыур лосгезоом мотугарбимоиокрзлиыелооиоб сб дебиеоизмуззыв овигелобеовоыакодыатробеазраникаон снешиыооварназо овоыакодыатрос зы виыоолхтиыоорисбоовотездизлеббртобимайг обеоизыасикртовзетоееорнаявлебиятовоеойагроагибок зыиырториз ыелоораовигикакуойабд навозм с логазыаевзмакуовыаовесообезлрт бб яовыаовесооренв яомыаоуоб зоисориз ыелетокареыоран вбяыозяозовыико овоевнареомыаорнегзы виыотаыоодыаобитугоорагатбаеосокогазыаевзмитоб та

In [82]:
text_accuracy(test_text, decoded_text)

0.25701943334790583

С попомью данного метода не получилось достичь приемлимого качества расшифровки за разумное количество итераций. Возможно стоит попробовать более простую функцию для вычисления логарифма правдоподобия и намного большее количество итераций.

### Расшифровка фрагмента

Попробуем расшифровать сообщение из задания

In [83]:
message =  "←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏"

In [86]:
decoder = MHDecoder(war_and_peace_prep, n=2, freq_vocab_size=200)
decoder.fit(message, 10_000)
print(decoder.num_accepted_steps)

true_to_enc = {v: k for k, v in decoder.enc_to_true.items()}
decoder_text = Encoder(true_to_enc, decoder.enc_to_true)
decoded_text = "".join(decoder_text.decode(split_n_grams(message, 1, 1)))

decoded_text

100%|██████████| 10000/10000 [04:10<00:00, 39.87it/s]

6883


'щдящйжчйжщкщтщййъъиюяцйчнйщящйюъфтщййъъиюяцйчнйтщвдтйжйцтъбъйдъъышщйщзйвътъъчнйящбвъйюъъфщтютцйдвъъщщйждщбъйжчйждщйдкщяюящйюъюжщяцйъйщйюъяжфщтщйиювдщиюяцйчнйыюяяйгюйюъдящкйщщйфщтжщътъщйгюкюйщщйвжъдюйщътзйвъйщфйъйзййщфщбъййщйъыщшюю'

In [90]:
decoder.fit(message, 1_000)

true_to_enc = {v: k for k, v in decoder.enc_to_true.items()}
decoder_text = Encoder(true_to_enc, decoder.enc_to_true)
decoded_text = "".join(decoder_text.decode(split_n_grams(message, 1, 1)))

decoded_text

100%|██████████| 1000/1000 [00:39<00:00, 25.49it/s]


'дьяжфчсфчжфжхдфькгхйяиьсбфжяжфдкьхжфькгхйяиьсбфхдцьхфяфахкпкфьккхэдьжмфцкхкгсбфядпцкфдгкьжхйхифьцкгддфчьдпкфчсфчьдфьфдяйяжфдгйчжяиькфжфдкяяьжхдфхйцьжхйяиьсбфхйяяфгйфдкьядфьддфьдхчдгхкдфгйфйьждфцягьйфнкхмфцкьдьькфмфьжьдпкфьдфкхдэйь'

С помощью этого метода не получилось расшифровать сообщение. Попробуем самый первый метод с сопоставлением частот букв

In [94]:
test_text_enc_list = list(message)
enc_freq = count_frequencies(test_text_enc_list)

enc_to_true, true_to_enc = fit_by_freq(true_words_freq, enc_freq)

decoder = Encoder(true_to_enc, enc_to_true)

decoded_text = "".join(decoder.decode(list(message)))
decoded_text

'осие кд кегено иоргиипидг еие подне иоргиипидг ноксн б цного сообжоиеб конордг иогко проденинп скороо ксого кд ксо сгоииие прикеипио е поибдено гиксегиипидг биии жи посиогиоо донкорноо жигииео кбрси цонб коиодио б иедого ио обожиц'

С помощью данного метода тоже не получилось расшифровать сообщение

Выводы: алгоритмы хорошие и задания интересные